# Load-Dependent Multi-Server PS with Two Classes

This example demonstrates load-dependent modeling with two job classes and PS scheduling.

## Features:
- Two closed classes: Class1 (4 jobs), Class2 (2 jobs)
- PS scheduling with load-dependent service capacity
- Servers available = min(total_jobs, c) where c=2
- Compares standard multi-server vs load-dependent behavior

## Load Dependence Matrix:
- Each class gets scaling based on total population
- Both classes benefit from additional server capacity when population increases

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.ctmc.*
import jline.util.matrix.*

In [ ]:
// Model parameters
val N = 4  // number of jobs in Class1
val c = 2  // number of servers

// Create standard multi-server network
val model = Network("model")
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)

// Create job classes
val jobclass1 = ClosedClass(model, "Class1", N, node1, 0)
val jobclass2 = ClosedClass(model, "Class2", N / 2, node1, 0)  // N/2 = 2 jobs

// Set service times
node1.setService(jobclass1, Exp.fitMean(1.0))   // Class1: mean = 1.0
node1.setService(jobclass2, Exp.fitMean(2.0))   // Class2: mean = 2.0
node2.setService(jobclass1, Exp.fitMean(1.5))   // Class1: mean = 1.5
node2.setService(jobclass2, Exp.fitMean(2.5))   // Class2: mean = 2.5
(node2 as Queue).numberOfServers = c

// Create routing matrix
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node1, 1.0)
model.link(P)

In [ ]:
// Solve standard model with MVA
println("=== Standard Multi-Server Model (MVA) ===")
val avgTableMva = MVA(model, "method", "exact").avgTable
avgTableMva.print()

In [ ]:
// Create load-dependent model
val ldmodel = Network("ldmodel")
val ldnode1 = Delay(ldmodel, "Delay")
val ldnode2 = Queue(ldmodel, "Queue1", SchedStrategy.PS)

// Create job classes for load-dependent model
val ldjobclass1 = ClosedClass(ldmodel, "Class1", N, ldnode1, 0)
val ldjobclass2 = ClosedClass(ldmodel, "Class2", N / 2, ldnode1, 0)

// Set service times for load-dependent model
ldnode1.setService(ldjobclass1, Exp.fitMean(1.0))
ldnode1.setService(ldjobclass2, Exp.fitMean(2.0))
ldnode2.setService(ldjobclass1, Exp.fitMean(1.5))
ldnode2.setService(ldjobclass2, Exp.fitMean(2.5))

// Set load dependence: min(total_jobs, c) servers available
val maxJobs = N + N / 2  // 4 + 2 = 6 total jobs
val ldMatrix = Array(maxJobs + 1) { totalJobs ->
    val serversAvailable = kotlin.math.min(kotlin.math.max(totalJobs, 1).toDouble(), c.toDouble())
    doubleArrayOf(serversAvailable, serversAvailable)  // Same scaling for both classes
}

// Convert to list format for setLoadDependence
val ldFunction = DoubleArray(maxJobs + 1) { i ->
    kotlin.math.min(kotlin.math.max(i, 1).toDouble(), c.toDouble())
}
ldnode2.setLoadDependence(ldFunction.toList())
(ldnode2 as Queue).numberOfServers = c

println("Load dependence function: ${ldFunction.toList()}")

// Create routing matrix for load-dependent model
val PLd = ldmodel.initRoutingMatrix()
PLd.set(ldjobclass1, ldjobclass1, ldnode1, ldnode2, 1.0)
PLd.set(ldjobclass1, ldjobclass1, ldnode2, ldnode1, 1.0)
PLd.set(ldjobclass2, ldjobclass2, ldnode1, ldnode2, 1.0)
PLd.set(ldjobclass2, ldjobclass2, ldnode2, ldnode1, 1.0)
ldmodel.link(PLd)

In [ ]:
// Try NC solver with different methods
println("\n=== NC Solver Results ===")
try {
    val lldAvgTableNC = NC(ldmodel).avgTable
    println("NC default:")
    lldAvgTableNC.print()
} catch (e: Exception) {
    println("NC default failed: ${e.message}")
}

val ncMethods = listOf("rd", "nrp", "nrl")
for (method in ncMethods) {
    try {
        val solver = NC(ldmodel, "method", method)
        val result = solver.avgTable
        println("\nNC $method:")
        result.print()
    } catch (e: Exception) {
        println("NC $method failed: ${e.message}")
    }
}

In [ ]:
// Try MVA solvers for load-dependent model
println("\n=== MVA Solver Results ===")
val mvaMethods = listOf("exact", "qd")
for (method in mvaMethods) {
    try {
        val solver = MVA(ldmodel, "method", method)
        val result = solver.avgTable
        println("\nMVA $method:")
        result.print()
    } catch (e: Exception) {
        println("MVA $method failed: ${e.message}")
    }
}

In [ ]:
// Try CTMC solver (may have issues with load dependence matrix)
println("\n=== CTMC Solver ===")
try {
    val lldAvgTableCTMC = CTMC(ldmodel).avgTable
    println("CTMC Results:")
    lldAvgTableCTMC.print()
} catch (e: Exception) {
    println("CTMC solver failed (expected for complex load dependence): ${e.message}")
}